# Conceptual Captions 12m. 
# Evaluating labels similarity score using Bottom-Up-Attention
https://ai.google.com/research/ConceptualCaptions/ - Dataset z 3.3m obrazow
    
https://github.com/MILVLG/bottom-up-attention.pytorch - Bottom-up-attention model ktory uzywamy

## 1. Preparations. Installing Packages. Download images. Cleaning up.

Zeby urochmic caly skrypt najpierw trzeba zainstalowac wszystkie niezbedne pakiety oraz biblioteki.
Dlatego przygotowalismy script zeby to wszystko zainstalowac.
Trzeba uruchomic **setup_all_libs.sh**, ktory znajduje sie w katalogu glownym, albo uruchomic komurke ponizej.
Na poczatku w katalogu glownym musza sie znajdowac ponizsze pliki i foldery:
#### 1) Image_Labels_Subset_Train_GCC-Labels-training.tsv
    (trzeba pobrac ze strony wyzej ConceptualCaptions)
#### 2) setup_all_libs.sh
#### 3) process_images_cpu.sh
#### 4) process_images_gpu.sh
#### 5) bottom-up-attention.pytorch/
#### 6) bottom-up-attention.pytorch/bua-d2-frcn-x152
    (trzeba pobrac ze strony wyzej bottom-up-attention 

Robimy importy oraz wczytujemy slowniki klas oraz atributow zeby potem rozpakowac wynikowy pliki ktore powstali po uruchamianiu modeli 

In [34]:
import os
import torch
import numpy as np
import pandas as pd

model_eval_path = './bottom-up-attention.pytorch/evaluation/'
model_path = './bottom-up-attention.pytorch/'

# Load classes
classes = ['__background__']
with open(os.path.join(model_eval_path, 'objects_vocab.txt')) as f:
    for object in f.readlines():
        classes.append(object.split(',')[0].lower().strip())

# Load attributes
attributes = ['__no_attribute__']
with open(os.path.join(model_eval_path, 'attributes_vocab.txt')) as f:
    for att in f.readlines():
        attributes.append(att.split(',')[0].lower().strip())

Wczytujemy labely w DataFrame oraz zmieniamy nazwy kolumn zeby w przyszlosci bylo wygodno poslugiwac sie DataFramem

In [35]:
sample = pd.read_csv('Image_Labels_Subset_Train_GCC-Labels-training.tsv', sep='\t')
index_arr = [0, 1, 2, 3, 4]
tmp = sample.columns.to_series(index=index_arr).values
sample.columns = index_arr

sample.loc[-1] = tmp
sample.index = sample.index + 1 
sample.sort_index(inplace=True)
sample.head()

,0,1,2,3,4
0,christmas tree on a black background .,https://thumb1.shutterstock.com/display_pic_wi...,"christmas tree,christmas decoration,font,text,...","/m/025nd,/m/05fc9mj,/m/03gq5hm,/m/07s6nbt,/m/0...","0.9818305373191833,0.952756941318512,0.9227379..."
1,item : drawing of a figure surrounded by person,https://i.pinimg.com/736x/f9/fd/48/f9fd4878090...,"drawing,modern art,line,visual arts,art,sketch...","/m/02csf,/m/015r61,/m/03scnj,/m/0p9xx,/m/0jjw,...","0.8945257067680359,0.8489813804626465,0.828646..."
2,the sidewalk near the corner of streets has on...,http://s3-us-west-2.amazonaws.com/ktoo/2017/08...,"mode of transport,transport,vehicle,street,nei...","/m/079bkr,/m/07bsy,/m/07yv9,/m/01c8br,/m/0180x...","0.8588771224021912,0.8513096570968628,0.783954..."
3,actor attends the season premiere,https://media.gettyimages.com/photos/aidan-gil...,"musician,premiere,event,singer,suit,performance","/m/09jwl,/m/03n3f3,/m/081pkj,/m/09l65,/m/01xyh...","0.8445718288421631,0.8338453769683838,0.670432..."
4,"another possible invitation -- love the font ,...",https://i.pinimg.com/736x/c1/c3/3a/c1c33aed458...,"text,font,material property,label","/m/07s6nbt,/m/03gq5hm,/m/0457gc6,/m/05c0n6k","0.9419509172439575,0.8490258455276489,0.699199..."


Tutaj sa metody dla sciagania obrazkow (**download_at_range**) oraz wyczyszczenia uszkodzonych obrazkow (**clear_damaged**)

In [36]:
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from multiprocessing.pool import ThreadPool
import requests
import cv2

def download_at_range(start, end):
    def download(data):
        path = os.path.join(model_path, 'image_labels/images/')
        index, url = data

        try:
            #print(index + '_start')
            response = requests.get(url)
        except requests.exceptions.ConnectionError:
            return None

        with open((path + str(index) + '.jpg'), 'wb') as handle:
            handle.write(response.content)
        #print(str(index) + '_finish')
        
    sliced_urls = sample[1].loc[start:end-1].values
    urls = []
    
    for i in range(len(sliced_urls)):
        urls.append([i+start, sliced_urls[i]])
    
    i = 0
    results = ThreadPool(12).imap_unordered(download, urls)

    


    
def clear_damaged(dir_to):
    directory = os.fsencode(dir_to)
    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        
        if filename.endswith('.jpg'):
            img = cv2.imread(os.path.join(dir_to,str(filename)))
            #print(filename, img)
            if img is None:
                os.remove(os.path.join(dir_to, filename))
                print("Something went wrong")

**Srednio wychodzi 7-8 min dla 1tys. obrazkow**    

(mialismy internet 10mb/s srednio)

In [28]:
download_at_range(0,30)

In [29]:
clear_damaged(os.path.join(model_path, 'image_labels/images/'))

Premature end of JPEG file


## 2) Evaluation. Unpacking results. Finding Semantic Similarity.

Po pobraniu obrazkow trzeba uruchomic model dla otrzymania labelow.
Zeby to zrobic trzeba uruchomic albo:
#### 1) process_images_cpu.sh 
    jezeli procesor jest uzywany
#### 2) process_images_gpu.sh
    jezeli jest uzywana karta graficzna i CUDA 10.2 i cuDNN 7.6 sa zainstalowane

#### Dalej usuwamy niepotrzebne kolumny i zostawiamy tylko nazwy obrazow oraz ich lables wyliczone przez Google Vision

In [37]:
sample = sample.drop([1, 3, 4], axis=1)
sample.head()

,0,2
0,christmas tree on a black background .,"christmas tree,christmas decoration,font,text,..."
1,item : drawing of a figure surrounded by person,"drawing,modern art,line,visual arts,art,sketch..."
2,the sidewalk near the corner of streets has on...,"mode of transport,transport,vehicle,street,nei..."
3,actor attends the season premiere,"musician,premiere,event,singer,suit,performance"
4,"another possible invitation -- love the font ,...","text,font,material property,label"


#### Dodajemy do DataFrame kolumne z labels otrzymana z wynikow modeli Bottom-Up-Attention

In [40]:
column_names = ['image name', 'label our base', 'label Google base']
base = pd.DataFrame(columns = column_names)

for i in range(13600):
    temp_path = model_path + 'image_labels/out_dir/' + str(i) + '.npz'

    if not os.path.isfile(temp_path):
        continue;
    
    test_load = np.load(temp_path, allow_pickle=True)

    obj = test_load['info'].tolist()['objects_id']
    atr = test_load['info'].tolist()['attrs_id']
    labels_our = []

    if type(classes[obj[0] + 2]) != type('str') or type(attributes[atr[0] + 2]) != type('str') or type(sample[2][i]) != type('str'):
        continue;
                
    for j in range(len(obj)):
        labels_our.append(attributes[atr[j] + 2] + " " + classes[obj[j] + 2])
        
    labels_google = sample[2][i].split(',')    
    
    base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
    
base.tail()

/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The fra

/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The fra

/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The fra

/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The fra

/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The fra

...

/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The fra

/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The fra

/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The fra

/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base = base.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_17687/3108746818.py:24: FutureWarning: The fra

,image name,label our base,label Google base
11576,multiple male caucasian hand gestures isolated...,"[up hands, raised hand, white hands, raised ha...","[leg, hand, finger, arm, human, joint, human b..."
11577,side profile of a man riding a bicycle with a ...,"[black wheel, black wheel, young woman, white ...","[bicycle, bicycle part, bicycle wheel, sitting..."
11578,latin pop artist arrives at awards held,"[brown hair, black eyes, brown hair, pink shir...","[hair, hairstyle, premiere, fashion, blond, si..."
11579,pictured is a promotional poster for the film,"[black background, black hair, white face, bla...",[poster]
11580,master making hairstyle for the brown - haired...,"[white wall, long hair, brown hair, black eye,...","[hair, face, eyebrow, forehead, hairstyle, lip..."


#### **Kod ponizej byl uzywany dla testowania przetworzania DataFrame**
    **Jego nie trzeba uruchamiac**

In [21]:
column_names = ['image name', 'label our base', 'label Google base']
base_test = pd.DataFrame(columns = column_names)

for i in range(7):
    temp_path = model_path + 'image_labels/out_dir/' + str(i) + '.npz'
    
    if not os.path.isfile(temp_path):
        continue;
    
    test_load = np.load(temp_path, allow_pickle=True)

    obj = test_load['info'].tolist()['objects_id']
    atr = test_load['info'].tolist()['attrs_id']
    labels_our = []

    if type(classes[obj[0] + 2]) != type('str') or type(attributes[atr[0] + 2]) != type('str') or type(sample[2][i]) != type('str'):
        continue;
                
    for j in range(len(obj)):
        labels_our.append(attributes[atr[j] + 2] + " " + classes[obj[j] + 2])
        
    labels_google = sample[2][i].split(',')    
    
    base_test = base_test.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
    
print(base_test.tail())

                                          image name  \
2  the sidewalk near the corner of streets has on...   
3                  actor attends the season premiere   
4  another possible invitation -- love the font ,...   
5               a woman walks her dog on the beach .   
6  using shrubs and hedges as fences is a very po...   

                                      label our base  \
2  [white building, black building, parked car, y...   
3  [dark eyes, open mouth, dark eye, smiling face...   
4  [white paper, blue writing, white paper, blue ...   
5  [white dog, white tail, sandy beach, white hea...   
6  [wooden bench, paved walkway, green leaves, bl...   

                                   label Google base  
2  [mode of transport, transport, vehicle, street...  
3  [musician, premiere, event, singer, suit, perf...  
4             [text, font, material property, label]  
5  [water, beach, sea, shore, ocean, canidae, dog...  
6  [garden, shrub, tree, natural landscape, grass..

/tmp/ipykernel_49271/1279681606.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base_test = base_test.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_49271/1279681606.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base_test = base_test.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_49271/1279681606.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  base_test = base_test.append({'image name' : sample[0][i], 'label our base' : labels_our, 'label Google base' : labels_google}, ignore_index = True)
/tmp/ipykernel_49271/1279681606

### Uzywamy biblioteki NLTK i Wu-Palmer Similarity, zeby obliczyc podobienstwo klas i atrybutow.
Jak działa podobieństwo **Wu & Palmer** ?
Oblicza relatedness, biorąc pod uwagę głębokość dwóch synsetów w taksonomii WordNet, wraz z głębokością **LCS** (Least Common Subsumer)
Wynik może być 0 < wynik < = 1 . Wynik nigdy nie może być zerowy, ponieważ głębokość LCS nigdy nie jest zerowa (głębokość korzenia taksonomii jest jedna).
Oblicza podobieństwo na podstawie tego, jak podobne są słowa i gdzie synsety występują względem siebie w drzewie hypernymów.
    
Jednym z podstawowych wskaźników używanych do obliczenia podobieństwa jest najkrótsza ścieżka odległości między dwoma Synsetami i ich wspólnym hipernymem.
WordNet to leksykalna baza danych, czyli słownik języka angielskiego, specjalnie zaprojektowany do przetwarzania języka naturalnego.

Synset jest specjalnym rodzajem prostego interfejsu, który jest obecny w NLTK do wyszukiwania słów w WordNet. Instancje synsetu to grupy synonimicznych słów, które wyrażają tę samą koncepcję. Niektóre słowa mają tylko jeden Synset, a niektóre kilka.
To bedzie sie przetwarzalo bardzo dlugo (ok. 1.5 godziny), ale daje ciekawe wyniki

In [28]:
import nltk
nltk.download('popular')
from nltk.corpus import wordnet as wn
from itertools import product

import time

start = time.time()

max_score_row = []
for i in range(len(base['image name'])):
    max_score_word = []
    
    for j in range(len(base['label Google base'][i])):
        max_score = []
        label_x = base['label Google base'][i][j]
        
        if len(label_x) == 2:
            word_x1 = label_x[0]
            word_x2 = label_x[1]
        else:
            word_x1 = ''
            word_x2 = label_x[0]
            
        sem_x1 = wn.synsets(word_x1)
        sem_x2 = wn.synsets(word_x2)
        for k in range(len(base['label our base'][i])):
            maxscore1 = 0
            maxscore2 = 0
            label_y = base['label our base'][i][k]
            
            word_y1 = label_x[0]
            word_y2 = label_x[1]
            
            sem_y1 = wn.synsets(word_y1)
            sem_y2 = wn.synsets(word_y2)
            
            
            for x,y in list(product(*[sem_x2,sem_y2])):
                score = x.wup_similarity(y) # Wu-Palmer Similarity
                maxscore2 = score if maxscore2 < score else maxscore2
               
            max_score.append(maxscore2)

        max_score_word.append(max(max_score))
        
    max_score_row.append(sum(max_score_word) / len(max_score_word))
    
print(max_score_row)

end = time.time()

print((end - start)/7)

[0.8888888888888892, 0.8888888888888892, 0.8925925925925927, 0.888888888888889, 0.8888888888888888, 0.8907407407407409, 0.8922222222222225, 0.8888888888888892, 0.8888888888888891, 0.8888888888888892, 0.88968253968254, 0.8888888888888892, 0.8962962962962963, 0.8896296296296299, 0.8896296296296299, 0.8888888888888892, 0.8888888888888891, 0.8888888888888892, 0.8944444444444446, 0.8925925925925927, 0.8958333333333333, 0.8907407407407409, 0.8888888888888892, 0.8888888888888892, 0.8944444444444445, 0.888888888888889, 0.8888888888888892, 0.8907407407407409, 0.8888888888888892, 0.8888888888888892, 0.8888888888888891, 0.8888888888888888, 0.8940740740740742, 0.8888888888888892, 0.8916666666666668, 0.8928571428571429, 0.8958333333333333, 0.8933333333333334, 0.8888888888888891, 0.8888888888888891, 0.8896296296296299, 0.8951851851851853, 0.8888888888888892, 0.8896296296296299, 0.8903703703703706, 0.8896296296296299, 0.8944444444444443, 0.8888888888888892, 0.8964646464646466, 0.8888888888888891, 0.8

Tablica powyzej jest tablica zawierajaca **srednie wartosci podobenstwa klas** pomiedzy labels ktore dostalismy z modelu i labels ktore zostali wygenegrowane przez Google Vision.

Nizej jest obliczona srednia wartosc podobenstwa pomiedzy wynikami naszego modelu bottom-up-attention oraz wynikami Google Vision

In [30]:
print(np.mean(max_score_row))

0.8892011090280527


### SpaCy Word2Vector Similarity using en_core_web_lg
    
**Word2Vec** - to ogólna nazwa zestawu modeli opartych na sztucznych sieciach neuronowych zaprojektowanych do uzyskiwania wektorowych reprezentacji słów w języku naturalnym. Służy do analizy semantyki języków naturalnych, opartej na semantyce dystrybucyjnej, uczeniu maszynowym i wektorowej reprezentacji słów.
Program działa w następujący sposób: word2vec przyjmuje dużą obudowę tekstową jako dane wejściowe i dopasowuje wektor do każdego słowa, podając współrzędne słów na wyjściu. Najpierw generuje słownik korpusu, a następnie oblicza reprezentację wektorową słów, "ucząc się" na tekstach wejściowych. Reprezentacja wektorowa opiera się na bliskości kontekstowej: słowa występujące w tekście obok tych samych słów (a zatem o podobnym znaczeniu) będą miały wektory bliskie (w odległości cosinusowej).

In [17]:
import spacy
nlp = spacy.load('en_core_web_lg')
result_arr = []

for _, row in base.iterrows():
    a = nlp(' '.join(row['label Google base']))
    b = nlp(' '.join(row['label our base']))

    a = nlp(' '.join([str(t) for t in a if (not t.is_stop) and (t.pos_ in ['NOUN', 'PROPN']) ]))
    b = nlp(' '.join([str(t) for t in b if (not t.is_stop) and (t.pos_ in ['NOUN', 'PROPN']) ]))
    
    print()
    result_arr.append(a.similarity(b))
    
print(np.mean(result_arr))

0.6345984350399273


Wyniki naszych pomiarów uzyskane przez nasz drugi model "SpaCy Word2Vector Similarity" są w 63.45% podobne do wyników modelu Google Vision. Oznacza to, że drugi wybrany przez nas model nie dość dokładnie identyfikuje obiekty w porównaniu z Google Vision, ale śą niezbyt złe.

Jeśli porównamy pierwszy i drugi model, możemy powiedzieć, że wyniki drugiego modelu nie są takie dokładne jak wyniki pierwszego modelu. Jest to związane ze sposobem porównywania wektorów cech.


Wyniki naszych pomiarów uzyskane przez Wu-Palmer Similarity są w 88.92% podobne do wyników modelu Google Vision. Oznacza to, że wybrany przez nas model dość dokładnie identyfikuje obiekty w porównaniu z Google Vision.

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
result_arr = []

for _, row in base.iterrows():
    a = nlp(' '.join(row['label Google base']))
    b = nlp(' '.join(row['label our base']))
    sentences = [a, b]

    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    embeddings = model.encode(sentences)
    print(embeddings)

    results = cosine_similarity(
        [embeddings[0]],
        embeddings[1:]
    )
    result_arr.append(results)
    
print(np.mean(result_arr))